# Eventalign_collapse dev

In [3]:
from collections import namedtuple, Counter
from itertools import islice
import pandas as pd
import random
from pycl.pycl import *
import glob
import numpy as np

# Mismatching model/reference kmers

In [48]:
import numpy as np

In [ ]:
fp = "/home/aleg/Analyses/Nanopore_yeast/nanopolish/nanopolish_read_index_no_sec.tsv"
max_res = 10000
res = 0
df_list = []
c_len_mismatch = Counter ()

lines = []
with open (fp) as f:
    header = f.readline().rstrip().split()
    line_tuple = namedtuple("line", header)
    
    prev_lt = None
    for l in f:
        ls = l.rstrip().split()
        lt = line_tuple (*ls)
                
        if prev_lt and lt.reference_kmer != lt.model_kmer:
            if lines:
                lines.append(lt)
            else:
                lines = [prev_lt, lt]
                
        elif lines:
            lines.append(lt)
            c_len_mismatch [len(lines)-2] += 1            
            
            if len(lines) > 5:
                df_list.append(pd.DataFrame(lines))
                res += 1
                if res == max_res:
                    break
                
            lines = []
        
        prev_lt = lt

In [58]:
df = pd.DataFrame(c_len_mismatch.most_common(), columns=["mismatch_len", "count"])
df.sort_values (by="mismatch_len", inplace=True)
display (df[df["count"]>1])

,mismatch_len,count
0,1,210661
1,2,7222
2,3,1848
3,4,852
4,5,422
5,6,225
6,7,115
7,8,63
9,9,29
8,10,30


In [79]:
for df in random.sample(df_list, 10):
    with pd.option_context("display.max_rows",4):
        display (df)

,contig,position,reference_kmer,read_name,strand,event_index,event_level_mean,event_stdv,event_length,model_kmer,model_mean,model_stdv,standardized_level,start_idx,end_idx
0,YBL027W,553,TAAAG,b7082ba8-2f15-4274-b089-c6885cdb8283,t,706,95.81,4.558,0.00232,TAAAG,95.02,3.38,0.21,7427,7434
1,YBL027W,553,TAAAG,b7082ba8-2f15-4274-b089-c6885cdb8283,t,707,112.14,3.203,0.00266,NNNNN,0.00,0.00,inf,7419,7427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,YBL027W,553,TAAAG,b7082ba8-2f15-4274-b089-c6885cdb8283,t,710,120.99,11.525,0.00299,NNNNN,0.00,0.00,inf,7382,7391
5,YBL027W,553,TAAAG,b7082ba8-2f15-4274-b089-c6885cdb8283,t,711,66.98,2.434,0.01162,TAAAG,95.02,3.38,-7.39,7347,7382


,contig,position,reference_kmer,read_name,strand,event_index,event_level_mean,event_stdv,event_length,model_kmer,model_mean,model_stdv,standardized_level,start_idx,end_idx
0,YDL081C,316,ATTAG,5188d1a4-4cd3-402d-9946-2132a1fabe66,t,603,94.34,1.576,0.00631,ATTAG,93.40,3.38,0.24,10927,10946
1,YDL081C,316,ATTAG,5188d1a4-4cd3-402d-9946-2132a1fabe66,t,604,112.10,3.754,0.00299,NNNNN,0.00,0.00,inf,10918,10927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,YDL081C,316,ATTAG,5188d1a4-4cd3-402d-9946-2132a1fabe66,t,607,132.52,3.272,0.00266,NNNNN,0.00,0.00,inf,10856,10864
5,YDL081C,316,ATTAG,5188d1a4-4cd3-402d-9946-2132a1fabe66,t,608,103.20,5.510,0.00598,ATTAG,93.40,3.38,2.47,10838,10856


,contig,position,reference_kmer,read_name,strand,event_index,event_level_mean,event_stdv,event_length,model_kmer,model_mean,model_stdv,standardized_level,start_idx,end_idx
0,YML081C-A,175,ATTGA,92fb3a37-ea51-423d-a7cc-0398fefe3c99,t,400,96.42,3.482,0.00266,ATTGA,87.77,3.09,2.60,14069,14077
1,YMR279C,1,TGTTT,7704f799-7db5-4c40-a1e9-87897b990f1f,t,262,100.26,2.891,0.00398,AAACA,98.87,4.39,0.28,35011,35023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,YMR279C,133,CAACC,7704f799-7db5-4c40-a1e9-87897b990f1f,t,34,88.97,2.273,0.01029,GGTTG,109.08,9.52,-1.87,39398,39429
230,YLR197W,12,GAATA,6dd44de5-4006-4961-bb68-c714c1823580,t,34,100.82,0.930,0.00199,GAATA,100.11,3.93,0.16,73964,73970


,contig,position,reference_kmer,read_name,strand,event_index,event_level_mean,event_stdv,event_length,model_kmer,model_mean,model_stdv,standardized_level,start_idx,end_idx
0,YMR122W-A,250,TTTAA,aecb1a36-563b-4c09-a03c-d9a95c7b22f5,t,610,87.41,1.832,0.00531,TTTAA,83.05,2.92,1.37,21877,21893
1,YMR122W-A,250,TTTAA,aecb1a36-563b-4c09-a03c-d9a95c7b22f5,t,611,103.20,3.151,0.00498,NNNNN,0.00,0.00,inf,21862,21877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,YMR122W-A,250,TTTAA,aecb1a36-563b-4c09-a03c-d9a95c7b22f5,t,614,116.50,9.524,0.01793,NNNNN,0.00,0.00,inf,21786,21840
5,YMR122W-A,250,TTTAA,aecb1a36-563b-4c09-a03c-d9a95c7b22f5,t,615,120.83,6.771,0.01892,TTTAA,83.05,2.92,11.90,21729,21786


,contig,position,reference_kmer,read_name,strand,event_index,event_level_mean,event_stdv,event_length,model_kmer,model_mean,model_stdv,standardized_level,start_idx,end_idx
0,YDL081C,310,TATTT,9f3c8a10-a625-4d9e-8bf3-51f10ffa62d0,t,808,91.09,2.554,0.00730,TATTT,89.99,7.32,0.13,16519,16541
1,YDL081C,310,TATTT,9f3c8a10-a625-4d9e-8bf3-51f10ffa62d0,t,809,112.93,3.713,0.02590,NNNNN,0.00,0.00,inf,16441,16519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,YDL081C,310,TATTT,9f3c8a10-a625-4d9e-8bf3-51f10ffa62d0,t,812,126.24,4.727,0.00232,NNNNN,0.00,0.00,inf,16414,16421
5,YDL081C,312,TTTGA,9f3c8a10-a625-4d9e-8bf3-51f10ffa62d0,t,813,90.10,4.489,0.00764,TTTGA,85.48,3.51,1.15,16391,16414


,contig,position,reference_kmer,read_name,strand,event_index,event_level_mean,event_stdv,event_length,model_kmer,model_mean,model_stdv,standardized_level,start_idx,end_idx
0,YDR418W,493,ACTAA,6f0fa8a9-4d10-48bf-8f64-e2b060de86cd,t,648,91.43,1.282,0.00266,ACTAA,84.82,3.21,1.87,11102,11110
1,YDR418W,493,ACTAA,6f0fa8a9-4d10-48bf-8f64-e2b060de86cd,t,649,107.68,2.250,0.00896,NNNNN,0.00,0.00,inf,11075,11102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,YDR418W,493,ACTAA,6f0fa8a9-4d10-48bf-8f64-e2b060de86cd,t,653,108.18,3.540,0.00465,NNNNN,0.00,0.00,inf,11016,11030
6,YDR418W,493,ACTAA,6f0fa8a9-4d10-48bf-8f64-e2b060de86cd,t,654,118.90,5.183,0.00498,ACTAA,84.82,3.21,9.63,11001,11016


,contig,position,reference_kmer,read_name,strand,event_index,event_level_mean,event_stdv,event_length,model_kmer,model_mean,model_stdv,standardized_level,start_idx,end_idx
0,YHR174W,1293,CACCA,c4e52171-d33b-4d6c-be9c-b890e8ccae9b,t,1410,80.27,2.237,0.01627,CACCA,76.90,3.45,0.88,6789,6838
1,YHR174W,1293,CACCA,c4e52171-d33b-4d6c-be9c-b890e8ccae9b,t,1411,96.90,5.245,0.01029,NNNNN,0.00,0.00,inf,6758,6789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,YHR174W,1293,CACCA,c4e52171-d33b-4d6c-be9c-b890e8ccae9b,t,1417,104.39,7.760,0.01195,NNNNN,0.00,0.00,inf,6628,6664
8,YHR174W,1293,CACCA,c4e52171-d33b-4d6c-be9c-b890e8ccae9b,t,1418,78.40,4.389,0.00564,CACCA,76.90,3.45,0.39,6611,6628


,contig,position,reference_kmer,read_name,strand,event_index,event_level_mean,event_stdv,event_length,model_kmer,model_mean,model_stdv,standardized_level,start_idx,end_idx
0,YDR158W,1092,ATTTA,19e13d16-8889-4cb6-b156-b4b65ec898e5,t,2033,77.12,1.498,0.00996,ATTTA,77.38,2.53,-0.09,11427,11457
1,YDR158W,1092,ATTTA,19e13d16-8889-4cb6-b156-b4b65ec898e5,t,2034,86.46,2.828,0.00232,NNNNN,0.00,0.00,inf,11420,11427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,YDR158W,1092,ATTTA,19e13d16-8889-4cb6-b156-b4b65ec898e5,t,2037,114.27,2.897,0.00299,NNNNN,0.00,0.00,inf,11370,11379
5,YDR158W,1092,ATTTA,19e13d16-8889-4cb6-b156-b4b65ec898e5,t,2038,120.07,7.533,0.01029,ATTTA,77.38,2.53,14.86,11339,11370


,contig,position,reference_kmer,read_name,strand,event_index,event_level_mean,event_stdv,event_length,model_kmer,model_mean,model_stdv,standardized_level,start_idx,end_idx
0,YCR012W,831,GTCGT,aa2b9e09-b170-439f-895a-d6bdb36ccf27,t,2518,82.75,2.572,0.02888,GTCGT,83.38,2.73,-0.21,50819,50906
1,YCR012W,831,GTCGT,aa2b9e09-b170-439f-895a-d6bdb36ccf27,t,2519,91.71,1.819,0.00266,NNNNN,0.00,0.00,inf,50811,50819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,YCR012W,831,GTCGT,aa2b9e09-b170-439f-895a-d6bdb36ccf27,t,2526,117.51,8.787,0.00432,NNNNN,0.00,0.00,inf,50632,50645
9,YCR012W,831,GTCGT,aa2b9e09-b170-439f-895a-d6bdb36ccf27,t,2527,83.52,2.336,0.01793,GTCGT,83.38,2.73,0.05,50578,50632


,contig,position,reference_kmer,read_name,strand,event_index,event_level_mean,event_stdv,event_length,model_kmer,model_mean,model_stdv,standardized_level,start_idx,end_idx
0,YFR042W,598,AATGA,14f50c2e-28da-4f6f-b794-14f9acea3599,t,1280,112.14,2.172,0.00266,AATGA,84.80,5.00,4.81,14132,14140
1,YJR029W,4909,AGTCC,50a7975a-dbcc-441c-9fa1-2d01b0265cef,t,1076,122.28,10.337,0.00598,GGACT,125.25,6.02,-0.44,9459,9477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,YJR029W,5258,GATTC,50a7975a-dbcc-441c-9fa1-2d01b0265cef,t,475,99.80,3.942,0.00266,GAATC,111.27,3.91,-2.62,19470,19478
603,YLR256W,573,TCCAG,775781fc-e08d-4490-8719-a26b41ab7e0f,t,7,76.78,5.122,0.00398,TCCAG,74.28,2.68,0.83,51143,51155


# Gaps in reference

In [1]:
from collections import namedtuple, Counter
from itertools import islice
import pandas as pd
from tqdm import tqdm

#fp = "data/nanopolish_reads.tsv"
fp = "/home/aleg/Analyses/Nanopore_yeast/nanopolish/nanopolish_read_index.tsv"
c_gap = Counter()

line_list = []
prev_lt = None
with open (fp) as f:
    header = f.readline().rstrip().split()
    line_tuple = namedtuple("line", header)
    
    for l in tqdm(f):
        ls = l.rstrip().split()
        lt = line_tuple (*ls)
        
        if prev_lt and prev_lt.contig == lt.contig and prev_lt.read_name == lt.read_name:
            gap_len = int(lt.position) - int(prev_lt.position)
            if gap_len > 1:
                c_gap [gap_len] += 1
                #display (pd.DataFrame([prev_lt, lt]))
        prev_lt = lt
        
df = pd.DataFrame(c_gap.most_common(), columns=["gap_len", "count"])
df.sort_values (by="gap_len", inplace=True)
display (df)

17650628it [00:52, 334130.92it/s]


,gap_len,count
0,2,90342
1,3,74066
2,4,30944
3,5,10386
4,6,3997
5,7,1786
6,8,871
7,9,477
8,10,227
9,11,156


# Proto mp version 

### helper fun

In [5]:
import sys
import os

def stderr_print (*args):
    """reproduce print with stderr.write
    """
    sys.stderr.write(" ".join(str(a) for a in args))
    sys.stderr.flush()

def access_dir (fn, **kwargs):
    """Check if the directory is readeable
    """
    return os.path.isdir (fn) and os.access (fn, os.R_OK),

def access_file (fn, **kwargs):
    """Check if the directory is readeable
    """
    return os.path.isfile (fn) and os.access (fn, os.R_OK),

class Read ():
    
    def __init__ (self, header, read_id, ref_id):
        self.header = header
        self.read_id = read_id
        self.ref_id = ref_id
        self.kmer_list = []
        
    def __repr__(self):
        """ Readable description of the object """
        m = "[Read] read_id: {} / ref_id: {} / length :{}".format (self.read_id, self.ref_id, len(self))
        return m

    def __len__ (self):
        return len (self.kmer_list)
        
    def add_line (self, line):
        self.kmer_list.append (line)
    
    def kmer_list_collapse (self):
        pass
    
    def make_str (self):
        self.str_read = "#\t{}\t{}\n".format (self.read_id, self.ref_id)
        for line in self.kmer_list:
            self.str_read += "{}\n".format("\t".join(line[0:4]))

### Main function 

In [6]:
# -*- coding: utf-8 -*-

#~~~~~~~~~~~~~~IMPORTS~~~~~~~~~~~~~~#
# Standard library imports
import multiprocessing as mp
from time import time
from collections import Counter, namedtuple

# Third party imports
import numpy as np

# Local imports
############## from  import stderr_print, recursive_file_gen, access_dir

#~~~~~~~~~~~~~~CLASS~~~~~~~~~~~~~~#
class Eventalign_collapse ():

    def __init__ (self, output_fn, input_fn=None, threads=4, max_reads=None, verbose=False):
        """
        """
        
        self.output_fn = output_fn
        self.input_fn = input_fn
        self.threads = threads-2 # Remove 2 threads for read and write
        self.max_reads = max_reads
        self.verbose = verbose

        # Init Multiprocessing variables
        manager = mp.Manager ()
        in_q = manager.Queue (maxsize = 10000)
        out_q = manager.Queue (maxsize = 10000)

        # Define processes
        read_ps = mp.Process (target=self._split_reads, args=(in_q,))
        work_ps_list = []
        for i in range (self.threads):
            work_ps_list.append (mp.Process (target=self._process_read, args=(in_q, out_q)))
        write_ps = mp.Process (target=self._write_output, args=(out_q,))

        # Start processes
        read_ps.start ()
        for ps in work_ps_list:
            ps.start ()
        write_ps.start ()

        # Join processes
        read_ps.join ()
        for ps in work_ps_list:
            ps.join ()
        write_ps.join ()

    #~~~~~~~~~~~~~~PRIVATE METHODS~~~~~~~~~~~~~~#
    def _split_reads (self, in_q):
        """
        Mono-threaded reader
        """
        
        try:
            # Open input file or stdin
            input_fp = open (self.input_fn, "r") if self.input_fn else sys.stdin
            
            # Get header
            header = input_fp.readline().rstrip().split("\t")

            # first line
            ls = input_fp.readline().rstrip().split("\t")
            read = Read (header=header, read_id=ls[3], ref_id=ls[0])
            read.add_line (ls)
            n_reads = 1
            
            for line in input_fp:

                # Early ending if required
                if self.max_reads and n_reads > self.max_reads:
                    break

                ls = line.rstrip().split("\t")

                # Line correspond to the same ids 
                if ls[3] == read.read_id and ls[0] == read.ref_id:
                    read.add_line (ls)

                # New ids = enqueue prev list and start new one
                else:
                    # Put read_list in queue and update counter 
                    in_q.put (read)
                    n_reads+=1

                    # Reset values
                    read = Read (header=header, read_id=ls[3], ref_id=ls[0])
                    read.add_line (ls)
                    
        except Exception as E:
            stderr_print (E)

        if self.input_fn:
            input_fp.close()
        
        # Add 1 poison pill for each worker thread
        for i in range (self.threads):
            in_q.put(None)

    def _process_read (self, in_q, out_q):
        """
        Multi-threaded workers
        """
        for read in iter(in_q.get, None):
            read.kmer_list_collapse ()
            read.make_str ()
            out_q.put (read)

        # Add poison pill in queues
        out_q.put (None)

    def _write_output (self, out_q):
        """
        Mono-threaded Writer
        """

        offset = 0
        n_reads = 0
        
        # Open output files 
        with open (self.output_fn, "w") as output_fp, open (self.output_fn+".idx", "w") as idx_fp:
            
            for _ in range (self.threads):
                for read in iter (out_q.get, None):

                    output_fp.write (read.str_read)
                    idx_fp.write ("{}\t{}\t{}\n".format (read.read_id, read.ref_id, offset))
                    offset += len(read.str_read)
                    n_reads += 1

                    stderr_print (f"{n_reads}\r")

In [26]:
#%%timeit -r 20
a = Eventalign_collapse (input_fn="./data/nanopolish_reads_index_samples.tsv", output_fn= "./output/test.tsv", threads=6, verbose=True, max_reads=100)

In [15]:
a = Eventalign_collapse (input_fn="../../../Analyses/Nanopore_yeast/nanopolish/nanopolish_read_index.tsv", output_fn= "./output/test.tsv", threads=6, verbose=True)

In [ ]:
a = Eventalign_collapse (input_fn="../../../Analyses/Nanopore_yeast/nanopolish/nanopolish_read_index.tsv", output_fn= "./output/test.tsv", threads=6, verbose=True)

In [16]:
head ("./output/test.tsv.idx", 25)

0	YGR240C	0
2	YHR174W	47493
1	YCR030C	80164
3	YHR174W	119845
4	YHR174W	160959
5	YLR441C	194528
6	YGR192C	213760
8	YDR500C	250256
9	YDR224C	259085
7	YGR192C	272438
11	YDL145C	311976
10	YIL117C	332779
14	YDR382W	377159
13	YPL198W	398745
15	YLR110C	429355
12	YLR293C	445540
16	YMR116C	505659
17	YEL052W	536794
18	YGL076C	557003
19	YKL060C	588117
Only 20 lines in the file


# Test package

In [1]:
from NanopolishComp.Eventalign_collapse import Eventalign_collapse

In [5]:
input_fn = "/home/aleg/Analyses/RNA_Yeast_TRM5/eventalign/KO/eventalign.tsv"
output_fn = "/home/aleg/Analyses/RNA_Yeast_TRM5/eventalign/KO/eventalign_collapsed.tsv"

a = Eventalign_collapse (input_fn=input_fn, output_fn=output_fn, verbose=True, threads=6)

head (output_fn, ignore_comment_line=True)
head (output_fn+".idx")

Collapse file by read_id/ref_id
12428 reads [05:22, 38.48 reads/s]
[Eventalign_collapse] total reads: 12428 [38.47 reads/s]


ref_pos ref_kmer n_events NNNNN_events mismatching_events start_idx end_idx mean      median     n_signals 
0       ATGTT    2        0            0                  18125     18148   90.034904 90.0864    23        
1       TGTTC    1        0            0                  18115     18125   105.92871 106.610504 10        
2       GTTCA    1        0            0                  18081     18115   72.699455 72.673294  34        
3       TTCAG    2        0            0                  18056     18081   75.84177  74.674     25        
4       TCAGC    1        0            0                  18043     18056   92.09281  92.4576    13        
5       CAGCG    1        0            0                  18025     18043   110.99039 111.4265   18        
6       AGCGA    1        0            0                  17997     18025   113.14164 114.020004 28        
7       GCGAA    3        0            0                  17947     17997   92.015976 93.1986    50        
8       CGAAT    1        0 

In [7]:
output_fn = "/home/aleg/Analyses/RNA_Yeast_TRM5/eventalign/KO/eventalign_collapsed.tsv"

# Import the index in a pandas dataframe (because it is simple)
import pandas as pd 
index_df = pd.read_table (output_fn+".idx")

# Select random lines
random_lines = index_df.sample(5)
print ("Random index lines")
display (random_lines)

# Open the collapsed event align file
with open (output_fn) as fp:
    for id, read in random_lines.iterrows():
        
        # Access the header corresponding to the randomly selected index line using seek 
        fp.seek(0) # Return to file start
        fp.seek(read.byte_offset) # Move to the offset indicated in the index file
        print (fp.readline().rstrip()) # Print read header
        df = pd.read_table (fp, nrows=read.kmers) # Read lines corresponding to the read
        with pd.option_context("display.max_rows",4): # display first and last lines
            display(df)

Random index lines


,ref_id,ref_start,ref_end,read_id,kmers,NNNNN_kmers,mismatching_kmers,missing_kmers,byte_offset,byte_len
3987,YHR174W,292,1271,1efd16c7-337c-4c99-ab45-92161d7caa96,932,30,0,47,125999887,45840
7124,YHR174W,1067,1283,e46103f8-b687-454b-a5f1-1058e478ab8f,208,6,0,8,212324659,10509
541,YDR091C,381,1793,5933681f-ac91-432d-a004-9c5cd765a4c3,1378,28,0,37,9025877,68154
994,YFR044C,953,1438,b4a5003a-66e2-4572-8528-745ab2ac6c1b,465,13,0,20,16869453,23247
10785,YGR254W,1154,1303,ddd9c5dc-029a-425c-a4ec-3cfbc9fbcd6f,123,9,0,26,276211209,6315


#1efd16c7-337c-4c99-ab45-92161d7caa96	YHR174W


,ref_pos,ref_kmer,n_events,NNNNN_events,mismatching_events,start_idx,end_idx,mean,median,n_signals
0,292,ATGGT,1,0,0,40706,40736,93.954475,93.794800,30
1,293,TGGTA,5,0,0,40609,40706,122.080550,122.095000,97
...,...,...,...,...,...,...,...,...,...,...
930,1269,GCTGT,9,0,0,9669,9765,77.661050,77.177246,96
931,1270,CTGTC,4,1,0,9608,9669,99.212260,96.987700,61


#e46103f8-b687-454b-a5f1-1058e478ab8f	YHR174W


...

,ref_pos,ref_kmer,n_events,NNNNN_events,mismatching_events,start_idx,end_idx,mean,median,n_signals
0,381,AAACC,1,0,0,71614,71621,105.152710,106.395000,7
1,382,AACCT,4,0,0,71531,71614,81.980156,81.495500,83
...,...,...,...,...,...,...,...,...,...,...
1376,1788,TCAGG,2,1,0,19586,19606,81.480810,81.421844,20
1377,1792,GAAAC,1,0,0,19578,19586,82.803116,82.674200,8


#b4a5003a-66e2-4572-8528-745ab2ac6c1b	YFR044C


,ref_pos,ref_kmer,n_events,NNNNN_events,mismatching_events,start_idx,end_idx,mean,median,n_signals
0,953,GTTGT,12,0,0,27602,27952,81.249115,81.14570,350
1,954,TTGTC,4,0,0,27550,27602,93.959870,94.89896,52
...,...,...,...,...,...,...,...,...,...,...
463,1436,AGAAA,4,0,0,11108,11143,129.329400,131.01000,35
464,1437,GAAAA,4,0,0,10933,11108,106.302110,107.32800,175


#ddd9c5dc-029a-425c-a4ec-3cfbc9fbcd6f	YGR254W


,ref_pos,ref_kmer,n_events,NNNNN_events,mismatching_events,start_idx,end_idx,mean,median,n_signals
0,1154,TGACT,5,1,0,22616,22716,114.341660,114.3600,100
1,1157,CTTGG,3,0,0,22567,22616,90.509346,90.9921,49
...,...,...,...,...,...,...,...,...,...,...
121,1301,TGACA,1,0,0,15515,15560,120.789350,123.9660,45
122,1302,GACAA,7,2,0,15388,15515,102.777170,107.3350,127
